# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))

# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace
try:
    query1 = """create keyspace if not exists sparkify """
    query2 = """with replication={'class':'SimpleStrategy','replication_factor':1}"""
    create_keyspace_query = query1 + query2
    session.execute(create_keyspace_query)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("sparkify")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# create table statement
create_q = """create table if not exists songs1 """
cols = """(session_id int, item_in_session int, artist text, song text, length float, """
pkey = """PRIMARY KEY (session_id, item_in_session))"""
query = create_q + cols + pkey

try:
    session.execute(query)
except Exception as e:
    print(e)         

In [ ]:
# insert related information into songs1 table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into songs1 (session_id, item_in_session, artist, song, length) "
        query = query + "values(%s, %s, %s, %s, %s)"
        v1, v2, v3, v4, v5 = int(line[8]), int(line[3]), line[0], line[9], float(line[5])
        session.execute(query, (v1, v2, v3, v4, v5))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
# do verification
query = """select artist, song, length from songs1 where session_id=338 and item_in_session=4"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

### Query 2

In [45]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# create table statement
create_q = """create table if not exists songs2 """
cols = """(user_id int, session_id int, item_in_session int, artist text, song text, first_name text, last_name text, """
pkey = """PRIMARY KEY ((user_id, session_id), item_in_session))"""
query = create_q + cols + pkey

try:
    session.execute(query)
except Exception as e:
    print(e)         

In [46]:
# insert related information into songs2 table

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into songs2 (user_id, session_id, item_in_session, artist, song, first_name, last_name) "
        query = query + "values(%s, %s, %s, %s, %s, %s, %s)"
        v1, v2, v3, v4, v5, v6, v7 = int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]
        session.execute(query, (v1, v2, v3, v4, v5, v6, v7))

In [47]:
# do verification
query = """select artist, song, first_name, last_name from songs2 where user_id=10 and session_id=182"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.first_name, row.last_name)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3 

In [31]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# create statement
create_q = """create table if not exists songs3 """
cols = """(first_name text, last_name text, song text, user_id int, """
pkey = """PRIMARY KEY (song, user_id))"""
query = create_q + cols + pkey

try:
    session.execute(query)
except Exception as e:
    print(e)         

Jacqueline Lynch
Tegan Levine
Sara Johnson


In [ ]:
# insert related information into songs3 table

file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## TO-DO: Assign the INSERT statements into the `query` variable
        query = "insert into songs3 (first_name, last_name, song, user_id) "
        query = query + "values(%s, %s, %s, %s)"
        v1, v2, v3, v4 = line[1], line[4], line[9], int(line[10])
        session.execute(query, (v1, v2, v3, v4))

In [ ]:
## do verification
query = """select first_name, last_name from songs3 where song='All Hands Against His Own'"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.first_name, row.last_name)

### Drop the tables before closing out the sessions

In [32]:
## Drop the table before closing out the sessions

try:
    rows = session.execute("drop table if exists songs1")
except Exception as e:
    print(e)

try:
    rows = session.execute("drop table if exists songs2")
except Exception as e:
    print(e)
    
try:
    rows = session.execute("drop table if exists songs3")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()